## 랩 1: 간단한 고객 지원 에이전트 프로토타입 생성

### 개요

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/)는 모든 프레임워크와 모델을 사용하여 AI 에이전트를 안전하게 대규모로 배포하고 운영할 수 있도록 도와줍니다. 프로토타입에서 프로덕션으로 더 빠르게 이동할 수 있는 기능을 제공합니다.

이 5개 랩 튜토리얼에서는 **고객 지원 에이전트**를 사용하여 프로토타입에서 프로덕션까지의 엔드투엔드 여정을 보여드리겠습니다. 이 예제에서는 에이전트 구축을 위한 간단하고 코드 우선 프레임워크인 [Strands Agents](https://strandsagents.com/latest/)와 Amazon Bedrock의 Anthropic Claude Sonnet 3.7 모델을 사용합니다. 애플리케이션에서는 원하는 프레임워크와 모델을 사용할 수 있습니다. 여기서 다루는 개념들은 다른 프레임워크와 모델에서도 적용할 수 있다는 점이 중요합니다.

**워크샵 여정:**
- **랩 1 (현재)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **랩 2**: 메모리로 향상 - 대화 컨텍스트와 개인화 추가
- **랩 3**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구 안전하게 공유
- **랩 4**: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore 런타임 사용
- **랩 5**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

이 첫 번째 랩에서는 워크샵 전반에 걸쳐 지속적인 메모리, 공유 도구, 완전한 관찰 가능성을 갖춘 여러 고객을 서비스하는 프로덕션 준비 시스템으로 발전할 고객 지원 에이전트 프로토타입을 구축합니다. 우리 에이전트는 다음과 같은 로컬 도구를 사용할 수 있습니다:
- **get_return_policy()** - 특정 제품의 반품 정책 가져오기
- **get_product_info()** - 제품 정보 가져오기
- **web_search()** - 문제 해결 도움을 위한 웹 검색


### 랩 1 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*로컬에서 실행되는 간단한 프로토타입. 후속 랩에서는 이를 공유 도구, 지속적인 메모리, 프로덕션급 관찰 가능성을 갖춘 AgentCore 서비스로 마이그레이션합니다.*

### 전제조건

* 적절한 권한을 가진 **AWS 계정**
* 로컬에 설치된 **Python 3.10+**
* 자격 증명으로 구성된 **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화된 **Anthropic Claude 3.7**
* 다음 셀에서 설치될 **Strands Agents** 및 기타 라이브러리

### 1단계: 종속성 설치 및 라이브러리 가져오기
시작하기 전에 이 랩에 필요한 전제조건을 설치해보겠습니다

In [ ]:
# Install required packages
%pip install -U -r requirements.txt -q

이제 필요한 라이브러리를 가져오고 boto3 세션을 초기화할 수 있습니다

In [ ]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool
from scripts.utils import put_ssm_parameter

In [ ]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### 2단계: 사용자 정의 도구 구현

다음으로 고객 지원 에이전트에 제공될 3개의 도구를 구현하겠습니다.

Strands Agent에서 도구를 정의하는 것은 매우 간단합니다. 함수에 `@도구` 데코레이터를 추가하고 함수의 docstring에 도구 설명을 제공하기만 하면 됩니다. Strands Agents는 함수 문서, 타입 및 인수를 사용하여 에이전트에 이 도구에 대한 컨텍스트를 제공합니다.


#### 도구 1: 반품 정책 가져오기

**목적:** 이 도구는 고객이 다양한 제품 카테고리의 반품 정책을 이해할 수 있도록 도와줍니다. 반품 기간, 조건, 절차, 환불 일정에 대한 자세한 정보를 제공하여 고객이 제품을 반품할 때 무엇을 기대할 수 있는지 정확히 알 수 있도록 합니다.

In [ ]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included"
        },
         "laptops": {
            "window": "30 days", 
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available"
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty"
        }
    }

    # Default policy for unlisted categories
    default_policy = {
        "window": "30 days",
        "condition": "Original condition with all included components",
        "process": "Contact technical support",
        "refund_time": "5-7 business days after inspection", 
        "shipping": "Return shipping policies vary",
        "warranty": "Standard manufacturer warranty applies"
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return f"Return Policy - {product_category.title()}:\n\n" \
           f"• Return window: {policy['window']} from delivery\n" \
           f"• Condition: {policy['condition']}\n" \
           f"• Process: {policy['process']}\n" \
           f"• Refund timeline: {policy['refund_time']}\n" \
           f"• Shipping: {policy['shipping']}\n" \
           f"• Warranty: {policy['warranty']}"
print("✅ Return policy tool ready")

#### 도구 2: 제품 정보 가져오기

**목적:** 이 도구는 보증, 사용 가능한 모델, 주요 기능, 배송 정책, 반품 정보를 포함한 포괄적인 제품 세부 사항을 고객에게 제공합니다. 고객이 정보에 기반한 구매 결정을 내리고 구매하는 제품을 이해할 수 있도록 도와줍니다.

In [ ]:
@tool
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included"
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included"
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app"
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support"
        }
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return f"Technical Information - {product_type.title()}:\n\n" \
           f"• Warranty: {product['warranty']}\n" \
           f"• Specifications: {product['specs']}\n" \
           f"• Key Features: {product['features']}\n" \
           f"• Compatibility: {product['compatibility']}\n" \
           f"• Support: {product['support']}"

print("✅ get_product_info tool ready")

#### 도구 3: 웹 검색

**목적:** 이 도구를 통해 고객은 문제 해결 지원이나 제품 추천 등에 대한 제안을 받을 수 있습니다.

In [ ]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

print("✅ Web search tool ready")

### 4단계: 고객 지원 에이전트 생성 및 구성

다음으로 모델, 이전 단계에서 구현한 도구 목록, 시스템 프롬프트를 제공하여 고객 지원 에이전트를 생성하겠습니다.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

SYSTEM_PROMPT = """You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications.
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications."""

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.3,
    region_name=region
)

# Create the customer support agent with all tools
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search # Tool 3: Access the web for updated information
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("Customer Support Agent created successfully!")

### 5단계: 고객 지원 에이전트 테스트

모든 도구가 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

#### 반품 확인 테스트

In [ ]:
response = agent("What's the return policy for my thinkpad X1 Carbon?")

#### 문제 해결 테스트

In [ ]:
response = agent("I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?")

## 🎉 랩 1 완료!

기능적인 고객 지원 에이전트 프로토타입을 성공적으로 생성했습니다! 달성한 내용은 다음과 같습니다:

- 3개의 사용자 정의 도구(반품 정책, 제품 정보, 웹 검색)를 갖춘 에이전트 구축
- 다중 도구 상호작용 및 웹 검색 기능 테스트
- 프로덕션 여정을 위한 기초 구축

### 현재 한계 (이것들을 해결할 예정입니다!)
- **단일 사용자 대화 메모리** - 로컬 대화 세션, 여러 고객에게는 여러 세션이 필요
- **세션으로 제한된 대화 기록** - 대화에서 장기 메모리나 세션 간 정보를 사용할 수 없음
- **도구 재사용성** - 도구를 다른 에이전트에서 재사용할 수 없음
- **로컬에서만 실행** - 확장 불가능
- **아이덴티티** - 사용자 및/또는 에이전트 아이덴티티나 액세스 제어 없음
- **관찰 가능성** - 에이전트 동작에 대한 제한적인 관찰 가능성
- **기존 API** - 고객 데이터를 위한 기존 엔터프라이즈 API에 액세스 불가

##### 다음 단계 [랩 2: 메모리 추가로 에이전트 개인화 →](lab-02-agentcore-메모리.ipynb)
